# Section 5.4 Lab: Cross-Validation and the Bootstrap

We explore the use of validation set approach in order to estimate the test error rates that result from fitting various linear models on the **Auto** dataset.

In [1]:
library(ISLR)
set.seed(1)

We begin using the `sample()` function to split the set of obs into two halves, by selecting a random subset of 196 obs out of the original 392 obs. We refer to these obs as the trainig set.

In [2]:
train = sample(392, 196)

In [3]:
lm.fit = lm(mpg ~ horsepower, data = Auto, subset = train)

In [4]:
attach(Auto)

In [8]:
mean((mpg - predict(lm.fit, Auto))[-train]^2) 

[1] 23.26601